In [1]:
import food

food.main()

,calcium_calories_100_pct_rdi,calcium_grams_100_pct_rdi,pct_rdi_per_calorie,calcium_pct_rdi,description,category
4803,122.701383,105.777055,2.557567,94.538462,"TOFU,SALTED&FERMENTED (FUYU),PREP W/CA SULFATE",TOFU
7247,201.560874,135.275754,2.011097,73.923077,"CHEESE,MOZZARELLA,NON-FAT",CHEESE
7390,242.143600,135.275754,1.723064,73.923077,"CHEESE,SWISS,LOW FAT",CHEESE
4799,275.988287,190.336750,1.878023,52.538462,"TOFU,RAW,FIRM,PREP W/CA SULFATE",TOFU
7217,279.245283,188.679245,2.470633,53.000000,"CHEESE,PAST PROCESS,CHEDDAR OR AMERICAN,FAT-FREE",CHEESE
4801,287.535145,60.918463,1.766776,164.153846,"TOFU,DRIED-FROZEN (KOYADOFU),PREP W/CA SULFATE",TOFU
182,310.640216,117.222723,1.607726,85.307692,"CHEESE,PARMESAN,DRY GRATED,RED FAT",CHEESE
7142,313.625000,81.250000,4.481804,123.076923,"SMELT,DRIED (ALASKA NATIVE)",SMELT
7342,323.099415,190.058480,1.634445,52.615385,"CHEESE,PAST PROCESS,SWISS,LOFAT",CHEESE
7310,342.105263,190.058480,1.563648,52.615385,"CHEESE,PAST PROCESS,AMERICAN,LOFAT",CHEESE


,choline_calories_100_pct_rdi,choline_grams_100_pct_rdi,pct_rdi_per_calorie,choline_pct_rdi,description,category
3749,169.144603,112.016293,4.873988,89.272727,"BEEF,VAR MEATS&BY-PRODUCTS,BRAIN,CKD,SIMMRD",BEEF
3753,169.395712,107.212476,12.136184,93.272727,"BEEF,VAR MEATS&BY-PRODUCTS,KIDNEYS,CKD,SIMMRD",BEEF
3754,222.972973,165.165165,36.167017,60.545455,"BEEF,VAR MEATS&BY-PRODUCTS,LIVER,RAW",BEEF
3756,230.263158,131.578947,38.993059,76.000000,"BEEF, VAR MEATS&BY-PRODUCTS, LIVER, CKD, PAN-F...",BEEF
7118,230.645161,221.774194,26.979332,45.090909,"Fish, whitefish, eggs (Alaska Native)",Fish
4365,234.776119,164.179104,5.666650,60.909091,"ROE,MIXED SPECIES,RAW",ROE
114,244.354839,80.645161,1.927222,124.000000,"EGG,YOLK,RAW,FROZEN",EGG
3755,246.596244,129.107981,33.885779,77.454545,"BEEF,VAR MEATS & BY-PRODUCTS,LIVER,CKD,BRSD",BEEF
5071,248.387097,177.419355,41.843339,56.363636,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,RAW",VEAL
113,255.645161,80.645161,1.875900,124.000000,"EGG,YOLK,RAW,FRSH",EGG


,copper_calories_100_pct_rdi,copper_grams_100_pct_rdi,pct_rdi_per_calorie,copper_pct_rdi,description,category
5071,10.619469,7.585335,41.843339,1318.333333,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,RAW",VEAL
3756,10.796545,6.169454,38.993059,1620.888889,"BEEF, VAR MEATS&BY-PRODUCTS, LIVER, CKD, PAN-F...",BEEF
5073,11.541528,5.980066,40.363358,1672.222222,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,CKD,PAN-FRIED",VEAL
5072,11.566265,6.024096,43.451560,1660.000000,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,CKD,BRSD",VEAL
3755,12.035287,6.301197,33.885779,1587.000000,"BEEF,VAR MEATS & BY-PRODUCTS,LIVER,CKD,BRSD",BEEF
3754,12.455151,9.226038,36.167017,1083.888889,"BEEF,VAR MEATS&BY-PRODUCTS,LIVER,RAW",BEEF
4456,13.746631,20.215633,35.076721,494.666667,"OYSTER,EASTERN,WILD,RAW",OYSTER
4459,13.920646,20.174849,34.087288,495.666667,"OYSTER,EASTERN,CANNED",OYSTER
870,15.913321,11.964903,36.133118,835.777778,"GOOSE,LIVER,RAW",GOOSE
4458,16.290131,11.890606,31.400978,841.000000,"OYSTER,EASTERN,WILD,CKD,MOIST HEAT",OYSTER


,iron_calories_100_pct_rdi,iron_grams_100_pct_rdi,pct_rdi_per_calorie,iron_pct_rdi,description,category
3762,42.424242,40.404040,7.985138,247.500000,"BEEF,VAR MEATS&BY-PRODUCTS,SPLEEN,RAW",BEEF
5083,43.399379,42.969683,6.964123,232.722222,"LAMB,VAR MEATS&BY-PRODUCTS,SPLEEN,RAW",LAMB
7060,58.260870,86.956522,2.099018,115.000000,"OOPAH (TUNICATE),WHL ANIMAL (ALASKA NATIVE)",OOPAH (TUNICATE)
3763,66.310976,45.731707,6.198816,218.666667,"BEEF,VAR MEATS&BY-PRODUCTS,SPLEEN,CKD,BRSD",BEEF
5084,72.614430,46.547711,4.597037,214.833333,"LAMB,VAR MEATS&BY-PRODUCTS,SPLEEN,CKD,BRSD",LAMB
7024,77.142857,69.498069,4.234409,143.888889,"Whale, beluga, meat, raw (Alaska Native)",Whale
870,78.414674,58.958402,36.133118,169.611111,"GOOSE,LIVER,RAW",GOOSE
863,80.183426,58.958402,36.301393,169.611111,"DUCK,DOMESTICATED,LIVER,RAW",DUCK
2614,80.645161,80.645161,5.202117,124.000000,"PORK,FRSH,VAR MEATS&BY-PRODUCTS,SPLEEN,RAW",PORK
2609,80.952381,95.238095,4.764435,105.000000,"PORK,FRSH,VAR MEATS&BY-PRODUCTS,LUNGS,RAW",PORK


,magnesium_calories_100_pct_rdi,magnesium_grams_100_pct_rdi,pct_rdi_per_calorie,magnesium_pct_rdi,description,category
6179,148.477905,68.739771,6.722880,145.476190,"WHEAT BRAN,CRUDE",WHEAT BRAN
7411,151.200000,168.000000,3.431571,59.523810,"SNAIL,RAW",SNAIL
3297,166.909091,54.545455,2.816304,183.333333,"SEAWEED,AGAR,DRIED",SEAWEED
6162,169.935980,53.777209,6.747548,185.952381,"RICE BRAN,CRUDE",RICE BRAN
5848,191.903808,84.168337,4.509092,118.809524,"COCOA,DRY PDR,UNSWTND",COCOA
5849,194.117647,88.235294,4.732369,113.333333,"COCOA,DRY PDR,UNSWTND,PROC W/ALKALI",COCOA
3601,194.748603,58.659218,3.187736,170.476190,"COTTONSEED FLR,LOFAT (GLANDLESS)",COTTONSEED FLR
3602,202.815789,55.263158,2.684116,180.952381,"COTTONSEED MEAL,PART DEFATTED (GLANDLESS)",COTTONSEED MEAL
3600,209.126214,58.252427,2.995940,171.666667,"COTTONSEED FLR,PART DEFATTED (GLANDLESS)",COTTONSEED FLR
4537,229.411765,176.470588,4.060177,56.666667,"CONCH,BAKED OR BROILED",CONCH


...

,v_b6_calories_100_pct_rdi,v_b6_grams_100_pct_rdi,pct_rdi_per_calorie,v_b6_pct_rdi,description,category
2832,63.511831,211.706102,10.489906,47.235294,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,RAW",BALSAM-PEAR (BITTER GOURD)
3321,71.578947,223.684211,12.841840,44.705882,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,CKD,BLD,...",BALSAM-PEAR (BITTER GOURD)
2833,76.052632,223.684211,12.086438,44.705882,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,CKD,BLD,...",BALSAM-PEAR (BITTER GOURD)
2973,90.666667,141.666667,6.424246,70.588235,"HORSERADISH-TREE LEAFY TIPS,RAW",HORSERADISH-TREE LEAFY TIPS
3396,109.795479,182.992465,7.021006,54.647059,"HORSERADISH-TREE,LEAFY TIPS,CKD,BLD,DRND,W/SALT",HORSERADISH-TREE
2974,109.795479,182.992465,7.021006,54.647059,"HORSERADISH-TREE,LEAFY TIPS,CKD,BLD,DRND,WO/SALT",HORSERADISH-TREE
6162,131.990172,41.769042,6.747548,239.411765,"RICE BRAN,CRUDE",RICE BRAN
3581,138.718070,40.208136,3.151613,248.705882,"PEPPERS,PASILLA,DRIED",PEPPERS
7135,186.562150,190.369541,3.479607,52.529412,"Elk, free range, roast, eye of round, raw (Sho...",Elk
4417,204.000000,188.888889,3.053314,52.941176,"TUNA,FRESH,YELLOWFIN,RAW",TUNA


,v_c_calories_100_pct_rdi,v_c_grams_100_pct_rdi,pct_rdi_per_calorie,v_c_pct_rdi,description,category
2185,1.293750,5.625000,78.951044,1777.777778,"ACEROLA JUICE,RAW",ACEROLA JUICE
2184,1.716738,5.364807,59.549235,1864.000000,"ACEROLA,(WEST INDIAN CHERRY),RAW",ACEROLA
1196,13.235294,120.320856,11.253490,83.111111,"SAUCE,RTS,PEPPER OR HOT",SAUCE
3552,13.242507,49.046322,9.622871,203.888889,"PEPPERS,SWEET,YELLOW,RAW",PEPPERS
3434,13.684211,52.631579,10.485579,190.000000,"PEPPERS,SWT,RED,CKD,BLD,DRND,W/SALT",PEPPERS
3433,14.736842,52.631579,9.736609,190.000000,"PEPPERS,SWT,RED,CKD,BLD,DRND,WO/SALT",PEPPERS
3301,14.845361,37.113402,9.766048,269.444444,"PEPPERS,HOT CHILI,GRN,RAW",PEPPERS
3279,14.873684,4.736842,9.984429,2111.111111,"PEPPERS,SWT,GRN,FREEZE-DRIED",PEPPERS
3536,14.873684,4.736842,11.188380,2111.111111,"PEPPERS,SWT,RED,FREEZE-DRIED",PEPPERS
3073,15.220588,66.176471,13.672892,151.111111,"POKEBERRY SHOOTS,(POKE),RAW",POKEBERRY SHOOTS


,v_e_calories_100_pct_rdi,v_e_grams_100_pct_rdi,pct_rdi_per_calorie,v_e_pct_rdi,description,category
4682,205.208333,34.722222,2.047980,288.000000,"PEANUT BUTTER,SMOOTH,VIT & MINERAL FORT",PEANUT BUTTER
4683,205.902778,34.722222,2.124303,288.000000,"PEANUT BUTTER,CHUNKY,VITAMIN&MINERAL FORT",PEANUT BUTTER
3616,244.426094,41.288192,1.956049,242.200000,"SUNFLOWER SD KRNLS,OIL RSTD,WO/SALT",SUNFLOWER SD KRNLS
3705,244.426094,41.288192,1.956049,242.200000,"SUNFLOWER SD KRNLS,OIL RSTD,W/SALT",SUNFLOWER SD KRNLS
3614,263.617213,45.139934,1.958127,221.533333,"SUNFLOWER SD KRNLS,DRIED",SUNFLOWER SD KRNLS
208,288.845727,83.240844,4.933914,120.133333,"GINGER,GROUND",GINGER
4365,306.428571,214.285714,5.666650,46.666667,"ROE,MIXED SPECIES,RAW",ROE
3615,334.482759,57.471264,1.894758,174.000000,"SUNFLOWER SD KRNLS,DRY RSTD,WO/SALT",SUNFLOWER SD KRNLS
3704,334.482759,57.471264,1.894758,174.000000,"SUNFLOWER SD KRNLS,DRY RSTD,W/SALT",SUNFLOWER SD KRNLS
655,378.551136,71.022727,1.867631,140.800000,"MARGARINE,MARGARINE-LIKE VEG OIL SPRD,60% FAT,TUB",MARGARINE


,v_k_calories_100_pct_rdi,v_k_grams_100_pct_rdi,pct_rdi_per_calorie,v_k_pct_rdi,description,category
2818,2.421053,10.526316,50.923601,950.000000,"AMARANTH LEAVES,RAW",AMARANTH LEAVES
2917,2.746988,14.457831,46.450332,691.666667,"CHARD,SWISS,RAW",CHARD
2985,4.081633,13.605442,31.024155,735.000000,"KALE,FRZ,CKD,BLD,DRND,WO/SALT",KALE
3401,4.081633,13.605442,31.024155,735.000000,"KALE,FRZ,CKD,BLD,DRND,W/SALT",KALE
2983,4.112607,14.687882,31.817852,680.833333,"KALE,CKD,BLD,DRND,WO/SALT",KALE
3252,5.280000,48.000000,32.993154,208.333333,"WATERCRESS,RAW",WATERCRESS
3462,5.591572,24.311183,29.532248,411.333333,"SPINACH,CKD,BLD,DRND,W/SALT",SPINACH
3162,5.591572,24.311183,29.532248,411.333333,"SPINACH,CKD,BLD,DRND,WO/ SALT",SPINACH
3161,5.715469,24.849865,28.619292,402.416667,"SPINACH,RAW",SPINACH
3164,5.979203,25.996534,25.939560,384.666667,"SPINACH,CND,DRND SOL",SPINACH


,zinc_calories_100_pct_rdi,zinc_grams_100_pct_rdi,pct_rdi_per_calorie,zinc_pct_rdi,description,category
4456,8.236978,12.113203,35.076721,825.545455,"OYSTER,EASTERN,WILD,RAW",OYSTER
4458,8.298001,6.056935,31.400978,1651.000000,"OYSTER,EASTERN,WILD,CKD,MOIST HEAT",OYSTER
4459,8.345245,12.094557,34.087288,826.818182,"OYSTER,EASTERN,CANNED",OYSTER
4533,10.760870,14.945652,33.965708,669.090909,"OYSTER,EASTERN,WILD,CKD,DRY HEAT",OYSTER
4534,17.114979,29.008439,22.416209,344.727273,"OYSTER,EASTERN,FARMED,RAW",OYSTER
4535,19.246955,24.363234,23.453839,410.454545,"OYSTER,EASTERN,FARMED,CKD,DRY HEAT",OYSTER
4457,24.870883,12.624813,11.165563,792.090909,"OYSTER,EASTERN,CKD,BREADED&FRIED",OYSTER
4460,53.610108,66.185319,15.783963,151.090909,"OYSTER,PACIFIC,RAW",OYSTER
4520,53.941035,33.092659,14.407508,302.181818,"OYSTER,PACIFIC,CKD,MOIST HEAT",OYSTER
1126,62.857143,130.952381,7.129092,76.363636,"SOUP,OYSTER STEW,CND,COND",SOUP
